### KNN - K nearest neighbors (Non parametric classification)

### 1. Numpy 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# function to read data and get features and labels
# arguments: filename
# return values: data frame with features and labels as last column
def readData (file):
    data = pd.read_csv (file)
    return data.iloc [:, 2:].values

In [3]:
# function to normalize data
# arguments: data
# return values: normalized data
def normalize (data):
    mins = np.min (data, axis = 0)
    maxs = np.max (data, axis = 0)
    return (data - mins)/ (maxs- mins)

In [4]:
# function split the dataset into train and test sets
# arguments: data
# return values: trainData, testData
def traintestsplit (data, split = 0.8):
    trainsize = int (len (data) * split)
    print ('Size of train data: ' + str (trainsize))
    print ('Size of test data: ' + str (len (data) - trainsize))
    np.random.shuffle (data)
    return data [:trainsize], data [trainsize:]

In [5]:
# calculate euclidean distance for two gievn points
# arguments: points (list)
# return values: euclidean distance (int)
def euclideanDistance (old, new):
    distance = 0
    for i in range (len (old)):
        distance += (old [i] - new [i]) ** 2
    return distance

In [6]:
# get k-nearest neighbors
# arguments: data, x , k
# return values: k-nearest neighbors
def knn (data, x, k):
    data = data.tolist ()
    for i in range (len (data)):
        compareVal = data [i] [:-1]
        distance = euclideanDistance (compareVal, x)
        data [i].append (distance)
    
    sortedByDistance = sorted (data, key = lambda x: (x [-1]))
    return sortedByDistance [:k]

In [7]:
# get decision for the class
# arguments: nearest, number of classes
# return values: class label
def classDecision (nearest, numClass):
    votes = [0] * numClass
    for i in range (len (nearest)):
        votes [int (nearest [i][-2])] += 1
    maxVal = max (votes)
    return votes.index (maxVal)

In [8]:
# loop through the test set and get prediction for each data point
# arguments: training data with labels, testing features, number of neighbors, number of classes
# return values: predictions for the features
def looper (trainData, testX, k, numClass):
    predictions = []
    textX = testX.tolist ()
    for row in testX:
        nearest = knn (trainData, row, k)
        predictions.append (classDecision (nearest, numClass))
        
    return predictions

In [9]:
data = readData ('./Data/Social_Network_Ads.csv')

In [10]:
datanorm = normalize (data [:, :-1])
labels = data [:, -1]
labels = np.reshape (labels, (len (labels), 1))

In [11]:
data = np.append (datanorm, labels, axis =1)

In [12]:
trainData, testData = traintestsplit (data)

Size of train data: 320
Size of test data: 80


In [13]:
testX, testY = data [:,:-1], data [:,-1]

In [14]:
# test for an example
nearest = knn (trainData, testX [150], 5)
len (nearest)

5

In [15]:
nearest

[[0.40476190476190477, 0.25925925925925924, 0.0, 0.0],
 [0.40476190476190477, 0.25925925925925924, 0.0, 0.0],
 [0.40476190476190477, 0.2814814814814815, 0.0, 0.0004938271604938286],
 [0.42857142857142855, 0.25925925925925924, 0.0, 0.0005668934240362798],
 [0.42857142857142855, 0.2740740740740741, 0.0, 0.0007863721620335369]]

In [16]:
label = classDecision(nearest, 2)

In [17]:
label

0

In [18]:
testY [150]

0.0

In [19]:
predictions = looper (trainData, testX, 5, 2)

In [20]:
from sklearn.metrics import confusion_matrix

In [21]:
cm = confusion_matrix (testY, predictions)

In [22]:
cm

array([[240,  17],
       [ 17, 126]], dtype=int64)

#### 2. Weighted K-nearest neighbors (robust)

Case 1: Suppose we have three classes [0, 1, 2]. We use k = 5 and get 2 votes for class 0, 2 votes for class 1 and 1 votes for class 2. Which one should be choose ? KNN model fails here

Case 2: Suppose we have two classes [0, 1]. We use k = 5 and get 3 votes for class 0 and 2 votes for class 1. But, class 1 points are much closer (lesser distance) to the given data point than class 0 points. KNN fairs here too.

Solution: Weighted KNN. We assign weights to each of the points. Instead of adding 1 to the value array, we had a decimal. 
We add 1/2 for the nearest neighbor's label, 1/3 for the second nearest and so on...

In [23]:
# get decision for the class
# arguments: nearest, number of classes
# return values: class label
def weightedClassDecision (nearest, numClass):
    votes = [0] * numClass
    for i in range (len (nearest)):
        votes [int (nearest [i][-2])] += 1/ (i + 2)
    maxVal = max (votes)
    return votes.index (maxVal)

In [24]:
# loop through the test set and get prediction for each data point
# arguments: training data with labels, testing features, number of neighbors, number of classes
# return values: predictions for the features
def weightedLooper (trainData, testX, k, numClass):
    predictions = []
    textX = testX.tolist ()
    for row in testX:
        nearest = knn (trainData, row, k)
        predictions.append (weightedClassDecision (nearest, numClass))
        
    return predictions

In [25]:
weightedPredictions = weightedLooper (trainData, testX, 5, 2)

In [26]:
cm = confusion_matrix (testY, weightedPredictions)

In [27]:
cm

array([[238,  19],
       [ 11, 132]], dtype=int64)